In [4]:
from huggingface_hub import HfApi
import os
api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="./dataset_storage/baby-lm-small.hf",
    repo_id="InoWouw/BabyLM-strict-small",
    repo_type="dataset",
)



data-00000-of-00012.arrow:   0%|          | 0.00/9.23M [00:00<?, ?B/s]









data-00000-of-00012.arrow:   1%|▏         | 123k/9.23M [00:00<00:08, 1.10MB/s]





data-00000-of-00012.arrow:   3%|▎         | 262k/9.23M [00:00<00:07, 1.25MB/s]







data-00000-of-00012.arrow:   4%|▍         | 393k/9.23M [00:00<00:07, 1.12MB/s]




data-00000-of-00012.arrow:   6%|▋         | 582k/9.23M [00:00<00:06, 1.33MB/s]






data-00000-of-00012.arrow:   8%|▊         | 721k/9.23M [00:00<00:07, 1.18MB/s]






data-00000-of-00012.arrow:   9%|▉         | 844k/9.23M [00:00<00:08, 1.04MB/s]


data-00000-of-00012.arrow:  10%|█         | 967k/9.23M [00:00<00:08, 963kB/s] 










data-00000-of-00012.arrow:  12%|█▏        | 1.10M/9.23M [00:01<00:08, 925kB/s]






data-00000-of-00012.arrow:  13%|█▎        | 1.23M/9.23M [00:01<00:08, 893kB/s]






data-00000-of-00012.arrow:  15%|█▍        | 1.36M/9.23M [00:01<00:08, 875kB/s]








data-00000-of-00012.arrow:  16%|█▌        | 1.50M/9.23M [00:01<00:08,

CommitInfo(commit_url='https://huggingface.co/datasets/InoWouw/BabyLM-strict-small/commit/4c83f40d5f07ef7e74b76a2bdb66b2372c68e93d', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4c83f40d5f07ef7e74b76a2bdb66b2372c68e93d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/InoWouw/BabyLM-strict-small', endpoint='https://huggingface.co', repo_type='dataset', repo_id='InoWouw/BabyLM-strict-small'), pr_revision=None, pr_num=None)